In [1]:
import sys
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSeg')

from ionpy.analysis import ResultsLoader
from ese.experiment.experiment import CalibrationExperiment
import seaborn as sns
sns.set_style("darkgrid")

import os 
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
))
os.environ["CUDA_VISIBLE_DEVICES"] = '0,1,2,3' 

%load_ext yamlmagic

%load_ext autoreload
%autoreload 

/storage/vbutoi/libraries/pylot/pandas/register.py:47: UserWarning: registration of accessor <class 'pylot.pandas.register.register_series_method.<locals>.inner.<locals>.AccessorMethod'> under name 'fillNA' for type <class 'pandas.core.series.Series'> is overriding a preexisting attribute with the same name.
  register_series_accessor(method.__name__)(AccessorMethod)
/storage/vbutoi/libraries/pylot/pandas/register.py:47: UserWarning: registration of accessor <class 'pylot.pandas.register.register_series_method.<locals>.inner.<locals>.AccessorMethod'> under name 'isNA' for type <class 'pandas.core.series.Series'> is overriding a preexisting attribute with the same name.
  register_series_accessor(method.__name__)(AccessorMethod)
/storage/vbutoi/libraries/pylot/pandas/register.py:47: UserWarning: registration of accessor <class 'pylot.pandas.register.register_series_method.<locals>.inner.<locals>.AccessorMethod'> under name 'notNA' for type <class 'pandas.core.series.Series'> is overridi

In [2]:
# Results loader object does everything
rs = ResultsLoader()
root = "/storage/vbutoi/scratch/ESE"

In [3]:
path = f"{root}/WMH_aug_runs"

dfc = rs.load_configs(
    path,
    properties=False,
)

df = rs.load_metrics(dfc)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [4]:
best_exp = rs.get_experiment(
    df=df,
    exp_class=CalibrationExperiment,
    metric="val-dice_score",
    checkpoint="max-val-dice_score",
    device="cuda"
)

/storage/vbutoi/projects/ionpy/util/libcheck.py:57: UserWarning: Using slow Pillow instead of Pillow-SIMD
  warn("Using slow Pillow instead of Pillow-SIMD")


In [5]:
%%yaml dataset_cfg 

dataset:
    _class: ese.experiment.datasets.WMH
    annotator: observer_o12
    axis: 0
    split: '?'
    slicing: full 
    task: '?'
    version: 0.2

<IPython.core.display.Javascript object>

In [6]:
dataset_options = {
    'dataset.split': ['train', 'val', 'cal'],
    'dataset.task': ['Singapore', 'Amsterdam']
}

In [7]:
from ionpy.util import dict_product, Config

base_cfg = Config(dataset_cfg)

cfgs = []
for cfg_update in dict_product(dataset_options):
    cfg = base_cfg.update(cfg_update)
    cfgs.append(cfg)

In [8]:
len(cfgs)

6

In [9]:
cfgs[0]

Config({'dataset': {'_class': 'ese.experiment.datasets.WMH', 'annotator': 'observer_o12', 'axis': 0, 'split': 'train', 'slicing': 'full', 'task': 'Singapore', 'version': 0.2}})

In [10]:
from ese.experiment.analysis.inference import get_dice_breakdown

get_dice_breakdown(best_exp, cfgs, num_bins=10)

Processing Singapore train


Subject Loop: 100%|██████████| 29/29 [18:07<00:00, 37.49s/it]


Processing Amsterdam train


Subject Loop: 100%|██████████| 41/41 [33:43<00:00, 49.35s/it]


Processing Singapore val


...

Subject Loop: 100%|██████████| 15/15 [10:45<00:00, 43.01s/it]


Processing Singapore cal


Subject Loop: 100%|██████████| 5/5 [03:31<00:00, 42.21s/it]


Processing Amsterdam cal


Subject Loop: 100%|██████████| 7/7 [06:58<00:00, 59.75s/it]
